In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [2]:
# import os
# os.chdir('/content/gdrive/MyDrive/Colab Notebooks/sequence_classification/longformer_zh')

In [3]:
ls

 ������ E �еľ��� m2
 �������к��� 6C3B-1FD3

 e:\NanZhang1991\NLP\sequence_classification\longformer_zh ��Ŀ¼

2022/08/18  08:07    <DIR>          .
2022/08/14  12:13    <DIR>          ..
2022/08/18  00:12    <DIR>          .ipynb_checkpoints
2022/07/10  22:15                 4 __init__.py
2022/07/10  22:15            19,059 classification.py
2022/08/18  02:23    <DIR>          finetune_model
2022/08/14  12:13    <DIR>          longformer
2022/08/18  02:13             9,937 longformer.ipynb
2022/08/18  21:34            56,635 longformer_suquence_classification.ipynb
2022/07/10  22:15             4,308 predict.py
2022/07/10  22:15             5,152 README.md
2022/07/10  22:15                75 requirements.txt
2022/08/18  21:33    <DIR>          runs
2022/07/18  02:41             5,157 test.ipynb
               8 ���ļ�        100,327 �ֽ�
               6 ��Ŀ¼ 206,425,665,536 �����ֽ�


## 导入环境

In [4]:
# ! pip install numpy pandas torch transformers progressbar tqdm

In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import json
import torch
from torch.utils.data import Dataset, DataLoader
# from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from progressbar import ProgressBar, Percentage, Bar, Timer, ETA, FileTransferSpeed
from transformers import BertTokenizer
from longformer.longformer import LongformerConfig, LongformerForSequenceClassification

d:\Program\minicoda3\envs\py3.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 构建数据集

### 数据清洗

In [5]:
fp = '../data/MultilabelSequenceClassification/chinese_dataset/train.zip'
train_df = pd.read_csv(fp).fillna(value='')
train_df.label = train_df.label.str.split('|')
train_df.sample(5)

,label,content
3915,[组织关系-退出],据天眼查数据显示，5月24日，上海证大喜马拉雅网络科技有限公司发生多项工商变更，小米旗下创投...
5140,[财经/交易-涨停],兴业矿业9月4日盘中涨停
3534,[司法行为-拘捕],双辽一男子酒驾被抓心怀怨恨“快手”评论区辱骂交警被拘留
8082,[竞赛行为-禁赛],昨天，根据英足总官方发布的最新消息，利物浦小将埃利奥特因为在2019赛季欧冠决赛后嘲讽热刺前...
9915,[竞赛行为-胜负],10月1日，中网ATP男子双打第一轮，中国选手公茂鑫、张哲组合在先失一盘的情况下，两下两盘，...


In [6]:
fp = '../data/MultilabelSequenceClassification/chinese_dataset/test.zip'
test_df = pd.read_csv(fp).fillna(value='')
test_df.label = test_df.label.str.split('|')
test_df.sample(5)

,label,content
649,[财经/交易-涨停],（10-10）涨停揭秘：基础化工板块走强震安科技涨停
944,[人生-死亡],陈乔恩想必大家都不陌生，在13年版的《笑傲江湖》饰演东方不败一角，凭借甜美的外表内在的淑女气...
677,[司法行为-罚款],英国信息监管局日前发表声明称，知名连锁酒店公司万豪国际集团2018年发生客户数据泄露事件，违...
787,[组织关系-辞/离职],中国国旅总经理彭辉辞职2018年薪酬为587万元
358,[产品行为-上映],科幻电影《上海堡垒》曝海报定档8月9日


### 标签id

In [7]:

def id2label(label):
    labels = label.explode().drop_duplicates()
    id2label = dict(zip(range(len(labels)), labels))
    label2id = {v: k for k, v in id2label.items()}

    with open("../data/MultilabelSequenceClassification/chinese_dataset/label.json", "w", encoding="utf-8") as f:
        f.write(json.dumps(id2label, ensure_ascii=False, indent=2))
    return id2label, label2id
    
id2label, label2id = id2label(train_df.label)
len(id2label)

65

In [9]:
def label_ids(label):
    label_ids = [0] * len(id2label)
    for separate_label in label:
        label_id = label2id.get(separate_label)
        if label_id is not None:
            label_ids[label_id] = 1
    return  label_ids

train_df['label_ids'] = train_df.label.apply(label_ids)
train_df.to_csv('../data/MultilabelSequenceClassification/chinese_dataset/train_dataset.zip')
train_df.head()

test_df['label_ids'] = test_df.label.apply(label_ids)
test_df.to_csv('../data/MultilabelSequenceClassification/chinese_dataset/test_dataset.zip')
test_df.head()

,label,content,label_ids
0,[组织关系-裁员],消失的“外企光环”，5月份在华裁员900余人，香饽饽变“臭”了,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[组织关系-裁员],前两天，被称为“仅次于苹果的软件服务商”的Oracle（甲骨文）公司突然宣布在中国裁员。。,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[组织关系-裁员],不仅仅是中国IT企业在裁员，为何500强的甲骨文也发生了全球裁员,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[组织关系-加盟, 组织关系-裁员]",据猛龙随队记者JoshLewenberg报道，消息人士透露，猛龙已将前锋萨加巴-科纳特裁掉。...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,[组织关系-裁员],冠军射手被裁掉，欲加入湖人队，但湖人却无意，冠军射手何去何从,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 模型训练

### 加载预训练模型

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
device

device(type='cpu')

In [10]:
# model_path = Path("../../models/distilbert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained(model_path)
# config = LongformerConfig.from_pretrained(model_path)
# config.problem_type = "multi_label_classification"
# config.num_labels = 6
# model = BertForSequenceClassification.from_pretrained(model_path, config=config)
# model.to(device)

### 加载数据集

In [11]:
import pandas as pd 
from ast import literal_eval

def load_dataset(train_path, train_size=0.9):
    df = pd.read_csv(train_path)
    new_df = df[['content', 'label_ids']].copy()
    new_df.rename(columns={'label_ids':'label'}, inplace=True)
    new_df.label = new_df.label.apply(literal_eval)
    train_data = new_df.sample(frac=train_size, random_state=200)
    val_data = new_df.drop(train_data.index)

    train_data.reset_index(drop=True, inplace=True)
    val_data.reset_index(drop=True, inplace=True)
    
    print(f"FULL Dataset: {new_df.shape}")
    print(f"TRAIN Dataset: {train_data.shape}")
    print(f"VALIDATION Dataset: {val_data.shape}")

    return train_data, val_data

# train_path = "../data/MultilabelSequenceClassification/chinese_dataset/train_dataset.zip"
# _train_data, _validation_data = load_dataset(train_path=train_path)

In [12]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['content']
        self.targets = self.data.label if "label" in  dataframe.columns else None
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer(
                                text,
                                None,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length',
                                return_token_type_ids=True,
                                truncation=True,
                                return_tensors='pt'
                               ).to(device)
        inputs_single = {}
        inputs_single['input_ids'] = inputs['input_ids'][0]
        inputs_single['attention_mask'] = inputs['attention_mask'][0]
        inputs_single['token_type_ids'] = inputs['token_type_ids'][0]
        if self.targets is not None:
            targets = torch.tensor(self.targets[index], dtype=torch.float).to(device)
        else:
            targets = torch.tensor([]).to(device)
        return inputs_single, targets

# train_dataset = CustomDataset(_train_data, tokenizer)

### 测试模型

In [13]:
def test_model(train_path, batch_size=16):
    train_data, validation_data = load_dataset(train_path)
    train_dataset = CustomDataset(train_data, tokenizer)
    validation_dataset = CustomDataset(validation_data, tokenizer )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(validation_dataset, batch_size=batch_size)
    
    for batch, data in enumerate(val_loader):
        print(batch, data)
        inputs, labels = data
        if batch == 1:
            break
    outputs = model(**inputs, labels=labels)
    print(outputs)

# test_model(train_path="../data/MultilabelSequenceClassification/chinese_dataset/train_dataset.zip")

In [14]:
# break

### 训练模型

In [15]:
# The Training Loop    
def train_one_epoch(model, train_loader, epoch_index, optimizer, tb_writer=None):
    # we’ll be using simple stochastic gradient descent with momentum

    
    running_loss = 0.
    last_loss = 0.
    # Add progress bar
    loop = tqdm(train_loader)

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(loop):
        
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(**inputs, labels=labels)
        # Gain loss
        loss = outputs.loss
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data 
        running_loss += loss.item()
        avg_loss = running_loss / (i + 1)
    
        # Finally, it reports the average per-batch loss for the last 1000 batches, for comparison with a validation run
        if (tb_writer) is not None  and (i % 1000 == 999):
            last_loss = running_loss / 1000 # loss per batch
            print('batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

        # Update progress bar
        loop.set_description(f"Epoch [{epoch_index}]") #
        loop.set_postfix(train_loss=avg_loss) 
        

    return avg_loss


In [16]:
# from transformers import BertTokenizer, LongformerConfig, LongformerForSequenceClassification

def train_model(train_path, max_len=1024, batch_size=16, epochs=10):

    model_path = Path("../../models/longformer_zh")
    tokenizer = BertTokenizer.from_pretrained(model_path)
    # config = LongformerConfig.from_pretrained(model_path)
    config = LongformerConfig.from_json_file(model_path /"config.json")
    # choose the attention mode 'n2', 'tvm' or 'sliding_chunks'
    # 'n2': for regular n2 attantion
    # 'tvm': a custom CUDA kernel implementation of our sliding window attention
    # 'sliding_chunks': a PyTorch implementation of our sliding window attention
    config.attention_mode = 'sliding_chunks'
    # If Use singe label
    # config.problem_type = "single_label_classification" If Use singe label
    config.problem_type = "multi_label_classification"
    config.num_labels = 65
    model = LongformerForSequenceClassification.from_pretrained(model_path, config=config)
    model = model.to(device)   

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    best_vloss = 1_000_000.

    train_data, validation_data = load_dataset(train_path, train_size=0.9)
    train_dataset = CustomDataset(train_data, tokenizer, max_len=max_len)
    validation_dataset = CustomDataset(validation_data, tokenizer, max_len=max_len)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    print(f"train_loader size {len(train_loader)}")
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

    # for name, param in model.named_parameters():
    #     if "layer.10" in name or "layer.11" in name:
    #         # print(name)
    #         param.requires_grad = True
    #     else:
    #         param.requires_grad = False

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
        
    for epoch in range(1, epochs+1):
        print(f'EPOCH: {epoch}')
        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(model, train_loader, epoch, optimizer, writer)

        widgets = ['Validation Progress: ', Percentage(), ' ', Bar('#'),' ', Timer(),
                ' ', ETA(), ' ', FileTransferSpeed()]
        pbar = ProgressBar(widgets=widgets, maxval=len(validation_loader)).start()
        # Validate the model. We don't need gradients on to do reporting
        with torch.no_grad():
            running_vloss = 0.0
            for batch, vdata in enumerate(validation_loader):
                vinputs, vlabels = vdata
                voutputs = model(**vinputs, labels=vlabels)
                vloss = voutputs.loss
                running_vloss += vloss
                pbar.update(batch + 1)
            avg_vloss = running_vloss / (batch + 1)
        print(f'LOSS train {avg_loss} valid {avg_vloss}')
        pbar.finish()
        
        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                           { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch)
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = f'finetune_model/multi_label.pt'
            torch.save(model.state_dict(), model_path)
        torch.cuda.empty_cache()
    print(f"best_vloss: {best_vloss}")



In [17]:
train_path = "../data/MultilabelSequenceClassification/chinese_dataset/train_dataset.zip"

EPOCHS = 5
MAX_LEN = 1024
BATCH_SIZE = 6
train_model(train_path, max_len=MAX_LEN, batch_size=BATCH_SIZE, epochs=EPOCHS)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LongformerTokenizer'. 
The class this function is called from is 'BertTokenizer'.


----------------- LongformerConfig {
  "_name_or_path": "..\\..\\models\\longformer_zh",
  "architectures": [
    "LongformerModel"
  ],
  "attention_dilation": null,
  "attention_mode": "sliding_chunks",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "autoregressive": false,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "directionality": "bidi",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "L

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at ..\..\models\longformer_zh and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FULL Dataset: (11958, 2)
TRAIN Dataset: (10762, 2)
VALIDATION Dataset: (1196, 2)
train_loader size 1794
EPOCH: 1


Epoch [1]:   1%|▏         | 26/1794 [34:19<39:16:52, 79.98s/it, train_loss=0.504]

In [ ]:
import pandas as pd

df =  pd.read_csv('../data/MultilabelSequenceClassification/chinese_dataset/train.zip')
df.head()
df_label = pd.read_json('../data/MultilabelSequenceClassification/chinese_dataset/label.zip', orient='index')
df_label.head()

,0
0,组织关系-裁员
1,司法行为-起诉
2,组织关系-解散
3,组织关系-加盟
4,司法行为-拘捕


In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
test_path = "../data/MultilabelSequenceClassification/toxic-comment-classification/test.csv.zip"
test_data = pd.read_csv(test_path)
test_dataset = CustomDataset(test_data, tokenizer, max_len=MAX_LEN)
print(f"TEST Dataset: {test_data.shape}")
test_loader = DataLoader(test_dataset, batch_size=16)  